In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import pandas
import numpy
import itertools

from explain_visuals import plot_original_overlap_counterfactual

import numpy as np
import pandas as pd
from explain_visuals import plot_original_overlap_counterfactual,plot_original_line_with_vals, plot_original_line_with_dots
import copy

In [ ]:
datafile = pandas.read_pickle("new_input.pkl")

In [ ]:
temp = datafile['wrapper_df']

In [ ]:
col_names = list(temp['combined_dat'].columns)

In [ ]:
display_names = {
 'hours_since_admit': 'Hours Since Admit',
 'hr': 'Heart rate',
 'rr': 'Respiratory rate',
 'sbp': 'SBP',
 'dbp': 'DBP',
 'o2sat': 'Oxygen saturation',
 'temp_c': 'Temperature (Celsius)',
 'avpu': 'AVPU mental status',
 'disoriented': 'Disoriented',
 'bmi': 'BMI',
 'fio2_final': 'Delivered FiO2',
 'braden_activity': 'Braden Scale - Activity',
 'braden_friction': 'Braden Scale - Friction',
 'braden_mobility': 'Braden Scale - Mobility',
 'braden_moisture': 'Braden Scale - Moisture',
 'braden_nutrition': 'Braden Scale - Nutrition',
 'braden_sensory': 'Braden Scale - Sensory',
 'braden_scale': 'Sum Total of Braden Scale',
 'urine_output_sum': 'Urine output sum over 24 hours',
 'albumin': 'Albumin',
 'alk_phos': 'Alkaline phosphatase',
 'anion_gap': 'Anion Gap',
 'bands_pct': 'Bands',
 'bili_total': 'Total bilirubin',
 'bun': 'BUN',
 'calcium': 'Calcium',
 'chloride': 'Chloride',
 'co2': 'CO2',
 'creatinine': 'Creatinine',
 'eosinophils_pct': 'Eosinophils',
 'gluc_ser': 'Glucose',
 'hb': 'Hemoglobin',
 'inr': 'INR',
 'lactate': 'Lactate',
 'lipase': 'Lipase',
 'lymphocytes_pct': 'Lymphocytes',
 'magnesium': 'Magnesium',
 'mcv': 'Mean Corpuscular Volume',
 'monocytes_pct': 'Monocytes',
 'neutrophils_pct': 'Neutrophils',
 'pco2_art': 'PCO2 (Arterial)',
 'pco2_ven': 'PCO2 (Venous)',
 'ph_art': 'pH (Arterial)',
 'ph_ven': 'pH (Venous)',
 'phosphate': 'Inorganic Phosphate',
 'platelet_count': 'Platelets',
 'po2_art': 'PO2 (Arterial)',
 'potassium': 'Potassium',
 'ptt': 'PTT',
 'rdw': 'RBC Distribution Width',
 'sgot': 'SGOT',
 'sodium': 'Sodium',
 'total_protein': 'Total Protein',
 'wbc': 'WBC count',
 'age': 'Age'
}

In [ ]:
id_to_int_map = {k: int(k+10000000) for k in temp['combined_dat']['encounter_id'].unique()}

def find_records(all_data, list_encids):
    return all_data[all_data['encounter_id'].isin(list_encids)]


In [ ]:
list_of_comte_folders = ['trial9_vis_outs/' + x for x in ['outs0']]

In [ ]:
list_of_anch_folders = ['trial8_vis_outs/' + x for x in ['outs0', 'outs1']]

In [ ]:
list_of_wshap_folders = ['trial9_vis_outs/' + x for x in ['outs0']]

In [ ]:
IS_WSHAP_DICT = False
ACTUAL_EXP_NAME = 'FeaturePerturbation' #WindowSHAP #ManualPermutation

# ACCROSS ALL COUNTS

In [ ]:
FOLDER_MAX_INT = 3
WINDOW_SHAP_N_TOP = 5
N_CASES = 3

In [ ]:
#load the explanations
across_all_counts = {'anchors':{}, 'comte':{}, 'windowshap':{}}
for foldernum, loc_folder in enumerate(list_of_comte_folders):
    for i in range(0, FOLDER_MAX_INT):
        
        ################################### Anchors ###################################
        anch_folder = list_of_anch_folders[foldernum]
        tf = open(f"{anch_folder}/Anchors_encounter_1000000{i}.0explanation.pkl", "rb")
        anchor = pickle.load(tf)[0]

        for acn in anchor['names']:
            f_name = acn.split(" ")[0]
            if f_name in across_all_counts['anchors'].keys():
                across_all_counts['anchors'][f_name] += 1
            else:
                across_all_counts['anchors'][f_name] = 1

        ################################### COMTE ###################################
        curid = i
        matching_records = find_records(temp['combined_dat'], [curid])
        matching_records['encounter_id'] = i
        orig = matching_records.to_numpy()

        tf = open(f"{loc_folder}/CoMTE_encounter_1000000{i}.0explanation.pkl", "rb")
        comte = pickle.load(tf)[0].squeeze()
        comte = numpy.concatenate([orig[:,:1], comte, orig[:,-1:]], axis=1)

        diffs = numpy.abs(numpy.sum(orig - comte, axis=0))
        for fid, f in enumerate(diffs):
            if f > 0.0001:
                if col_names[fid] in across_all_counts['comte'].keys():
                    across_all_counts['comte'][col_names[fid]] += 1
                else:
                    across_all_counts['comte'][col_names[fid]] = 1


        ################################### WINDOWSHAP ###################################
        wshap_folder = list_of_wshap_folders[foldernum]
        tf = open(f"{wshap_folder}/{ACTUAL_EXP_NAME}_encounter_1000000{i}.0explanation.pkl", "rb")
        if IS_WSHAP_DICT:
            wshap = pickle.load(tf)['wshap'][0]
        else:
            wshap = pickle.load(tf)[0]
            wshap = wshap - wshap.mean(axis=0)
        wshap = numpy.abs(wshap)
        tot_wshap = wshap.mean(axis=0)

        top10 = numpy.argpartition(tot_wshap, -WINDOW_SHAP_N_TOP)[-WINDOW_SHAP_N_TOP:]
        for top_id in top10:
            if col_names[top_id] in across_all_counts['windowshap'].keys():
                    across_all_counts['windowshap'][col_names[top_id]] += 1
            else:
                across_all_counts['windowshap'][col_names[top_id]] = 1

In [ ]:
#formatting explanation objects
wsc = across_all_counts['windowshap']
ws_counts = {k:v for k,v in sorted(wsc.items(), key=lambda pair: pair[1])}

cc = across_all_counts['comte']
c_counts = {k:v for k,v in sorted(cc.items(), key=lambda pair: pair[1])}

new_anchors = {}
for lbl,v in across_all_counts['anchors'].items():
    terms = "_".join(lbl.split("_")[:-1])
    if terms in new_anchors.keys():
        new_anchors[terms] += v
    else:
        new_anchors[terms] = v
a_count = {k:v for k,v in sorted(new_anchors.items(), key=lambda pair: pair[1])}

# ACROSS CASE COUNT

In [ ]:
#loading explanations for cases
across_case_counts = {f'case_{j}':{'anchors':{}, 'comte':{}, 'windowshap':{}} for j in range(0,N_CASES)}
for foldernum, loc_folder in enumerate(list_of_comte_folders):
    for i in range(0, N_CASES):
        ################################### Anchors ###################################
        anch_folder = list_of_anch_folders[foldernum]
        tf = open(f"{anch_folder}/Anchors_encounter_1000000{i}.0explanation.pkl", "rb")
        anchor = pickle.load(tf)[0]

        for acn in anchor['names']:
            f_name = acn.split(" ")[0]
            f_name = "_".join(f_name.split("_")[:-1])
            if f_name not in ['encounter_id', 'cuis']:
                if f_name in across_case_counts[f'case_{i}']['anchors'].keys():
                    across_case_counts[f'case_{i}']['anchors'][f_name] += 1
                else:
                    across_case_counts[f'case_{i}']['anchors'][f_name] = 1

        ################################### COMTE ###################################
        curid = i
        matching_records = find_records(temp['combined_dat'], [curid])
        matching_records['encounter_id'] = i
        orig = matching_records.to_numpy()

        tf = open(f"{loc_folder}/CoMTE_encounter_1000000{i}.0explanation.pkl", "rb")
        comte = pickle.load(tf)[0].squeeze()
        comte = numpy.concatenate([orig[:,:1], comte, orig[:,-1:]], axis=1)

        diffs = numpy.abs(numpy.sum(orig - comte, axis=0))
        for fid, f in enumerate(diffs):
            if f > 0.0001:
                if col_names[fid] not in ['encounter_id', 'cuis']:
                    if col_names[fid] in across_case_counts[f'case_{i}']['comte'].keys():
                        across_case_counts[f'case_{i}']['comte'][col_names[fid]] += 1
                    else:
                        across_case_counts[f'case_{i}']['comte'][col_names[fid]] = 1

        ################################### WINDOWSHAP ###################################
        wshap_folder = list_of_wshap_folders[foldernum]
        tf = open(f"{wshap_folder}/{ACTUAL_EXP_NAME}_encounter_1000000{i}.0explanation.pkl", "rb")
        if IS_WSHAP_DICT:
            wshap = pickle.load(tf)['wshap'][0]
        else:
            wshap = pickle.load(tf)[0]
            wshap = wshap - wshap.mean(axis=0)
        wshap = numpy.abs(wshap)[:, 3:-2]
        tot_wshap = wshap.sum(axis=0)

        top10 = numpy.argpartition(tot_wshap, -10)[-10:]
        for top_id in top10:
            if col_names[1:-1][top_id] in across_case_counts[f'case_{i}']['windowshap'].keys():
                    across_case_counts[f'case_{i}']['windowshap'][col_names[1:-1][top_id]] += 1
            else:
                across_case_counts[f'case_{i}']['windowshap'][col_names[1:-1][top_id]] = 1

In [ ]:
for case_id in range(0,N_CASES):
    case_counts = across_case_counts[f'case_{case_id}']
    matches = []
    dub_matches = set()
    m_list = ['anchors','comte','windowshap']
    for k in range(len(m_list)):
        method_exp = m_list.pop(0)
        case_method_vars = case_counts[method_exp].keys()
        other_methods = [x for x in m_list if x != method_exp]
        for v_name in case_method_vars:
            for other_method in other_methods:
                if v_name in case_counts[other_method].keys():
                    if v_name in matches:
                        dub_matches.add(v_name)
                        matches.remove(v_name)
                    elif v_name not in dub_matches:
                        matches.append(v_name)
    singles = [x for x in matches if x not in dub_matches]
    print(f"Case {case_id} - In Three: {dub_matches} - In Two: {matches}")

# PER CASE BEST OVERLAPPING EXAMPLES

In [ ]:
#loading best explanations

by_case_counts = {}
for casenum in range(0,N_CASES):
    ml = {}
    for methodtype in ['anchors','comte','windowshap']:
        lf = {}
        for f_num, c_id in enumerate(list_of_comte_folders):
            if methodtype == 'anchors':
                a_id = list_of_anch_folders[f_num]
                lf[a_id] = set()
            elif methodtype == 'windowshap':
                w_id = list_of_wshap_folders[f_num]
                lf[w_id] = set()
            else:
                lf[c_id] = set()
        ml[methodtype] = lf
    by_case_counts[casenum] = ml

filtered_col_names2 = copy.deepcopy(col_names)
filtered_col_names2.remove("encounter_id")
filtered_col_names2.remove("cuis")
col_name_to_id = {v:k for k,v in enumerate(filtered_col_names2)}

for i in range(0, N_CASES):
    for foldernum, loc_folder in enumerate(list_of_comte_folders):
        ################################### Anchors ###################################
        anch_folder = list_of_anch_folders[foldernum]
        tf = open(f"{anch_folder}/Anchors_encounter_1000000{i}.0explanation.pkl", "rb")
        anchor = pickle.load(tf)[0]

        ex_name = loc_folder
        for acn in anchor['names']:
            f_name = acn.split(" ")[0]
            f_name = "_".join(f_name.split("_")[:-1])
            if f_name not in ['encounter_id', 'cuis', 'hours_since_admit', 'timeofday', 'age']:
                by_case_counts[i]['anchors'][anch_folder].add(f_name)

        ################################### COMTE ###################################
        curid = i #int_to_id_map[i]
        matching_records = find_records(temp['combined_dat'], [curid])
        matching_records['encounter_id'] = i
        orig = matching_records.to_numpy()

        tf = open(f"{loc_folder}/CoMTE_encounter_1000000{i}.0explanation.pkl", "rb")
        comte = pickle.load(tf)[0].squeeze()
        comte = numpy.concatenate([orig[:,:1], comte, orig[:,-1:]], axis=1)

        diffs = numpy.abs(numpy.sum(orig - comte, axis=0))
        for fid, f in enumerate(diffs):
            if f > 0.0001:
                if col_names[fid] not in ['encounter_id', 'cuis', 'hours_since_admit', 'timeofday', 'age']:
                    by_case_counts[i]['comte'][ex_name].add(col_names[fid])

        ################################### WINDOWSHAP ###################################
        wshap_folder = list_of_wshap_folders[foldernum]
        tf = open(f"{wshap_folder}/{ACTUAL_EXP_NAME}_encounter_1000000{i}.0explanation.pkl", "rb")
        if IS_WSHAP_DICT:
            wshap = pickle.load(tf)['wshap'][0]
        else:
            wshap = pickle.load(tf)[0]
            wshap = wshap - wshap.mean(axis=0)
        wshap = numpy.abs(wshap)[:, 2:-2]
        tot_wshap = wshap.sum(axis=0)

        top10 = numpy.argpartition(tot_wshap, -10)[-10:]
        additional_loc = 1
        drop_ids = [col_name_to_id[x] for x in ['encounter_id', 'cuis', 'hours_since_admit', 'timeofday', 'age'] if x in col_name_to_id.keys()]
        for f_to_drop in ['encounter_id', 'cuis', 'hours_since_admit', 'timeofday', 'age']:
            if f_to_drop in col_name_to_id.keys():
                fd_id = col_name_to_id[f_to_drop]
                if fd_id in top10: 
                    new_val = numpy.argpartition(tot_wshap, -10-additional_loc)[-10-additional_loc]
                    additional_loc = additional_loc + 1
                    while new_val in top10 or new_val in drop_ids:
                        new_val = numpy.argpartition(tot_wshap, -10-additional_loc)[-10-additional_loc]
                        additional_loc = additional_loc + 1

                    top10[top10 == fd_id] = new_val                    
        
        for top_id in top10:
            if col_names[1:-1][top_id] not in ['encounter_id', 'cuis', 'hours_since_admit', 'timeofday', 'age']:
                by_case_counts[i]['windowshap'][wshap_folder].add(col_names[1:-1][top_id])

# Get the maximum overlapping cases

In [ ]:
all_best_combs = []
for caseid, mvals in by_case_counts.items():
    max_intersect_size = 0
    best_secondary_intersects = 0
    best_total_set_size = 999
    best_comb = None
    possibles = []
    best_anchor_coverage = 0
    
    all_anch_coverages = []
    for mname, explanationX in mvals.items():
        v_tups = []
        for tmpid, val in list(explanationX.items()):
            v_tups.append((tmpid, val))
        possibles.append(v_tups)
        
        if mname == 'anchors':
            for case_loc in list_of_anch_folders:
                tf = open(f"{anch_folder}/Anchors_encounter_1000000{i}.0explanation.pkl", "rb")
                anchor = pickle.load(tf)
                all_anch_coverages.append(anchor[0]['coverage'][-1])
            
    every_combo = list(itertools.product(*possibles))
    for comb in every_combo:
        comb[0][1].discard('outcome_ward24hr')
        comb[1][1].discard('outcome_ward24hr')
        comb[2][1].discard('outcome_ward24hr')
        total_intersect = comb[0][1].intersection(comb[1][1]).intersection(comb[2][1])
        total_union = comb[0][1].union(comb[1][1]).union(comb[2][1])
        secondary_intersect_count = len(comb[0][1].intersection(comb[1][1])) + len(comb[0][1].intersection(comb[2][1])) + len(comb[2][1].intersection(comb[1][1])) - (3 * len(total_intersect))        
        
        loc_id = comb[0][0].split("/")[1][4:]
        if loc_id == '':
            loc_id = 0
        else:
            loc_id = int(loc_id)
        anchor_coverage = all_anch_coverages[loc_id]
        
        if len(total_intersect) > max_intersect_size:
            best_comb = comb
            max_intersect_size = len(total_intersect)
            best_total_set_size = len(total_union)
            best_secondary_intersects = secondary_intersect_count
            best_anchor_coverage = anchor_coverage
        elif (len(total_intersect) == max_intersect_size) and (secondary_intersect_count > best_secondary_intersects):
            best_comb = comb
            max_intersect_size = len(total_intersect)
            best_total_set_size = len(total_union)
            best_secondary_intersects = secondary_intersect_count
            best_anchor_coverage = anchor_coverage
        elif (len(total_intersect) == max_intersect_size) and (secondary_intersect_count == best_secondary_intersects) and (len(total_union) < best_total_set_size):
            best_comb = comb
            max_intersect_size = len(total_intersect)
            best_total_set_size = len(total_union)
            best_secondary_intersects = secondary_intersect_count
            best_anchor_coverage = anchor_coverage
        elif (len(total_intersect) == max_intersect_size) and (secondary_intersect_count == best_secondary_intersects) and (len(total_union) == best_total_set_size) and anchor_coverage > best_anchor_coverage:
            best_comb = comb
            max_intersect_size = len(total_intersect)
            best_total_set_size = len(total_union)
            best_secondary_intersects = secondary_intersect_count
            best_anchor_coverage = anchor_coverage
            
    print(f"Case {caseid} best comb {best_comb} with primary {max_intersect_size} secondary {best_secondary_intersects} total_len {best_total_set_size}\n")
    all_best_combs.append(best_comb) 
    
    

# CHECKING CASES

In [ ]:
#for manual validation of cases

filtered_col_names = copy.deepcopy(col_names)
filtered_col_names.remove("encounter_id")
filtered_col_names.remove("cuis")
col_name_to_id = {v:k for k,v in enumerate(filtered_col_names)}
print(col_name_to_id)


how_many_window_shap_fs = 5


for i, case_data in enumerate(all_best_combs):
    anchor_folder = case_data[0][0]
    comte_folder = case_data[1][0]
    windowshap_folder = case_data[2][0]

    anch_fs = case_data[0][1]
    comt_fs = case_data[1][1]
    wshp_fs = case_data[2][1]
    print(f"SAFE CHECK: {anch_fs}")
    
    curid = i
    matching_records = find_records(temp['combined_dat'], [curid])
    orig = matching_records.to_numpy()[:, 1:-1]
    orig_time = matching_records['hours_since_admit'].to_numpy()
    print(f"ORIG SHAPE: {orig.shape}")
    print(f"ORIG TIMES: {orig_time}")
    
    #############################################################################################
    tf = open(f"{anchor_folder}/Anchors_encounter_1000000{i}.0explanation.pkl", "rb")
    anchor = pickle.load(tf)[0]
    

    #############################################################################################
    tf = open(f"{comte_folder}/CoMTE_encounter_1000000{i}.0explanation.pkl", "rb")
    comte = pickle.load(tf)[0].squeeze()
    print(f"COMTE SHAPE: {comte.shape}")
    
    #############################################################################################
    tf = open(f"{windowshap_folder}/{ACTUAL_EXP_NAME}_encounter_1000000{i}.0explanation.pkl", "rb")
    if IS_WSHAP_DICT:
        wshap = pickle.load(tf)['wshap'][0]
    else:
        wshap = pickle.load(tf)[0]
        wshap = wshap - wshap.mean(axis=0)
    
    print(f"wshap: {wshap.shape}")
    tot_wshap = wshap.sum(axis=0) / wshap.sum()
    print(f"wshap: {wshap.shape} tot_wshap:{tot_wshap.shape} numpy.argsort(tot_wshap):{numpy.argsort(tot_wshap)}")
    W_feat_sorted = np.expand_dims(numpy.argsort(tot_wshap), axis=0)[::-1]
    
    which_wshap_in_others = []
    print(f"CHECKING wshp_fs: {wshp_fs}")
    for feat in wshp_fs:
        if feat in anch_fs.union(comt_fs):
            which_wshap_in_others.append(feat)
    check_idx = 0
    list_wshap_fs = list(wshp_fs)
    while len(which_wshap_in_others) < how_many_window_shap_fs:
        if list_wshap_fs[check_idx] not in which_wshap_in_others:
            which_wshap_in_others.append(list_wshap_fs[check_idx])
        check_idx += 1
    final_windowshap_names = which_wshap_in_others
    
    #############################################################################################
    all_fs = anch_fs.union(comt_fs).union(set(final_windowshap_names))
    subset_orig = []
    all_fs_ids = []
    print(f"ALL FEATURES: {all_fs}")
    print(f"ALL FEATURES A: {anch_fs}")
    print(f"ALL FEATURES C: {comt_fs}")
    print(f"ALL FEATURES W: {final_windowshap_names}")
    
    for feat in all_fs:
        f_id = col_name_to_id[feat]
        all_fs_ids.append(f_id)
        subset_orig.append(orig[:, f_id])
    final_orig = np.stack(subset_orig, axis=1)
    
    comte_orig_data = []
    comte_fs_names = []
    comte_cf = []
    for comte_f in comt_fs:
        f_id = col_name_to_id[comte_f]
        comte_fs_names.append(comte_f)
        comte_orig_data.append(orig[:, f_id])
        comte_cf.append(comte[:, col_names.index(comte_f)])
    print(f"TESTING: {comte.shape} vs {len(col_names)} vs {orig.shape}")
    new_c = np.stack(comte_cf, axis=1)
    new_comte_o = np.stack(comte_orig_data, axis=1)

    ############################################################################################
    final_windowshap = []
    for f in all_fs:
        try:
            final_windowshap.append(wshap[:,col_name_to_id[f]])
        except:
            print(f"failed on {f} wshap: {wshap.shape} \n colnames: {col_name_to_id}")
    final_windowshap = np.stack(final_windowshap)    
    
    #############################################################################################
    anch_expl = np.zeros_like(orig)
    print(f"ANCHOR: {anchor['names']} against matching: {anch_fs}")
    filtered_anchor = []
    for feat_sign_val in anchor['names']:
        feat_time_name, sign, val = feat_sign_val.split(" ")
        f_name = "_".join(feat_time_name.split("_")[:-1])
        if f_name not in ['cuis', 'encounter_id', 'outcome_ward24hr', 'hours_since_admit', 'timeofday', 'age']:
            timepoint = feat_time_name.split("_")[-1]
            filtered_anchor.append(f"{f_name} at hour {int(timepoint)} {sign} {val}")
            
            if f_name not in anch_fs:
                raise NotImplementedError(f"ANCHOR FAIL ON {f_name} NOT IN {anch_fs}")
            fid = list(all_fs).index(f_name)
            anch_expl[fid, int(timepoint)] = val
    
    
    #############################################################################################
    new_c = np.expand_dims(new_c, axis=0)
    new_w = np.expand_dims(final_windowshap.transpose(), axis=0)
    allvals = temp['combined_dat'][list(all_fs)]

    print(f"PRINT comte shape: {comte.shape} and final: {final_orig.shape} "\
          f" and new_c_min_val: {new_c.shape} and {new_w.shape}" \
          f"{len(comte_fs_names)} and {len(all_fs)}")
    min_max_dict = {}
    
    for f_id, f_name in enumerate(list(all_fs)):
        if f_name in comte_fs_names:
            c_id = comte_fs_names.index(f_name)
            cf_min_val = new_c[:, :, c_id].min()
            cf_max_val = new_c[:, :, c_id].max()
        else:
            cf_min_val = 9999999
            cf_max_val = -1
            
        orig_min_val = final_orig[:, f_id].min()
        orig_max_val = final_orig[:, f_id].max()
                
        overall_max = max(orig_max_val, cf_max_val)
        overall_min = min(orig_min_val, cf_min_val)
        range_change = overall_max - overall_min
        overall_max = overall_max + (0.10 * range_change)
        overall_min = overall_min - (0.10 * range_change)

        print(f"MAXES: {orig_max_val} vs {cf_max_val} and {orig_min_val} vs  {cf_min_val}")
        
        min_max_dict[f_name] = (overall_min, overall_max)
    print(f"PRINT MINMAX_DICT: {min_max_dict}\n {new_comte_o}\n")
    plot_original_overlap_counterfactual(np.expand_dims(new_comte_o, axis=0), 
                                         [new_c], 
                                         comte_fs_names, 
                                         'viz_outs/', 
                                         image_name_prefix=f"cf_{i}_", 
                                         n_plots_horiz=1,
                                         min_max_dict=min_max_dict,
                                         display_names=display_names,
                                         timepoints=orig_time)
    
    plot_original_line_with_vals(np.expand_dims(final_orig, axis=0), 
                                 new_w, 
                                 list(all_fs), 
                                 'viz_outs/', 
                                 image_name_prefix=f"ws_{i}_", 
                                 n_plots_horiz=1,
                                 min_max_dict=min_max_dict,
                                 display_names=display_names,
                                 timepoints=orig_time)
    
    with open(f'viz_outs/ac_{i}.txt', "w") as anchor_file:
        subtext = ' AND\n'.join(["\t"+x for x in filtered_anchor])
        formatted_text = f"BECAUSE: \n {subtext} \nTHEN:\n\tThis patient is expected to be at high risk of clinical deterioration."
        anchor_file.write(formatted_text)
    anchor_file.close()
    print("--------------------------")
    



# Write synthetic data files

In [ ]:
per_case_superset = {}
for i, case_data in enumerate(all_best_combs):
    superset = set()
    for item in case_data:
        superset = superset.union(item[1])
    per_case_superset[i] = superset

combo_set = set()
for k,v in per_case_superset.items():
    combo_set = combo_set.union(v)
    
print(per_case_superset)
print("---------------------")
print(combo_set)
print(len(combo_set))

In [ ]:
vitals_disp = {
 'hours_since_admit': 'Hours Since Admit',
 'hr': 'Heart rate',
 'rr': 'Respiratory rate',
 'sbp': 'SBP',
 'dbp': 'DBP',
 'o2sat': 'Oxygen saturation',
 'temp_c': 'Temperature (Celsius)',
 'avpu': 'AVPU mental status',
 'disoriented': 'Disoriented',
 'bmi': 'BMI',
 'fio2_final': 'Delivered FiO2',
 'braden_activity': 'Braden Scale - Activity',
 'braden_friction': 'Braden Scale - Friction',
 'braden_mobility': 'Braden Scale - Mobility',
 'braden_moisture': 'Braden Scale - Moisture',
 'braden_nutrition': 'Braden Scale - Nutrition',
 'braden_sensory': 'Braden Scale - Sensory',
 'braden_scale': 'Sum Total of Braden Scale',
 'urine_output_sum': 'Urine output sum over 24 hours',
}
labs_disp = {
 'hours_since_admit': 'Hours Since Admit',
 'albumin': 'Albumin',
 'alk_phos': 'Alkaline phosphatase',
 'anion_gap': 'Anion Gap',
 'bands_pct': 'Bands',
 'bili_total': 'Total bilirubin',
 'bun': 'BUN',
 'calcium': 'Calcium',
 'chloride': 'Chloride',
 'co2': 'CO2',
 'creatinine': 'Creatinine',
 'eosinophils_pct': 'Eosinophils',
 'gluc_ser': 'Glucose',
 'hb': 'Hemoglobin',
 'inr': 'INR',
 'lactate': 'Lactate',
 'lipase': 'Lipase',
 'lymphocytes_pct': 'Lymphocytes',
 'magnesium': 'Magnesium',
 'mcv': 'Mean Corpuscular Volume',
 'monocytes_pct': 'Monocytes',
 'neutrophils_pct': 'Neutrophils',
 'pco2_art': 'PCO2 (Arterial)',
 'pco2_ven': 'PCO2 (Venous)',
 'ph_art': 'pH (Arterial)',
 'ph_ven': 'pH (Venous)',
 'phosphate': 'Inorganic Phosphate',
 'platelet_count': 'Platelets',
 'po2_art': 'PO2 (Arterial)',
 'potassium': 'Potassium',
 'ptt': 'PTT',
 'rdw': 'RBC Distribution Width',
 'sgot': 'SGOT',
 'sodium': 'Sodium',
 'total_protein': 'Total Protein',
 'wbc': 'WBC count',
 'age': 'Age'
}

In [ ]:
inv_map = {v:k for k,v in datafile['orig_dat_map'].items()}

In [ ]:
temp_nan_placeholder = -98989898989
for i, case_data in enumerate(all_best_combs):
    
    anchor_folder = case_data[0][0]    
    comte_folder = case_data[1][0]
    windowshap_folder = case_data[2][0]

    anch_fs = case_data[0][1]
    comt_fs = case_data[1][1]
    wshp_fs = case_data[2][1]
    
    curid = i
    matching_records = find_records(datafile['orig_dat_noise'], [inv_map[i]])
    print(f"MATCH SIZE: {matching_records.shape}")
    orig = pd.DataFrame(matching_records.to_numpy()[:, 1:-1], columns=filtered_col_names)
    orig = orig.fillna(temp_nan_placeholder)
    
    #do rounding
    mins = ((orig['hours_since_admit'] - np.floor(orig['hours_since_admit'])) * 60).round(decimals=2).astype(int)
    hours = np.floor(orig['hours_since_admit']).astype(int)
    strs = [f"{hours[i]}h {mins[i]}m" for i in range(mins.shape[0])]
    orig['hours_since_admit'] = strs
    orig['temp_c'] = np.floor(orig['temp_c'])
    orig['bmi'] = np.floor(orig['bmi'])
    orig['mcv'] = orig['mcv'].astype(float).round(2)
    orig['ptt'] = orig['ptt'].astype(float).round(2)
    orig['rdw'] = orig['rdw'].astype(float).round(2)
    
    #replace nan with str
    orig[orig == temp_nan_placeholder] = '-'
    
    subset = copy.deepcopy(orig)
    subset_labs = copy.deepcopy(orig)
    subset_vits = copy.deepcopy(orig)
    
    orig = orig.transpose()
    orig.to_csv(f"viz_outs/synthetic_case_{i}.csv", header=False)
    print(f"Case {i}: \n{orig}")
    
    used_cols = ['hours_since_admit'] + list(combo_set)
    subset = subset[used_cols]
    subset = subset.transpose()
    subset.to_csv(f"viz_outs/synthetic_anysubset_{i}.csv", header=False)
    
    #############################################
    used_cols_labs = ['hours_since_admit'] + [x for x in list(combo_set) if x in labs_disp.keys()]
    subset_labs = subset_labs[used_cols_labs]
    subset_labs.columns = [labs_disp[x] for x in used_cols_labs]
    subset_labs = subset_labs.transpose()
    subset_labs.to_csv(f"viz_outs/synthetic_labssubset_{i}.csv", header=False)
    
    used_cols_vits = ['hours_since_admit'] + [x for x in list(combo_set) if x in vitals_disp.keys()]
    subset_vits = subset_vits[used_cols_vits]
    subset_vits.columns = [vitals_disp[x] for x in used_cols_vits]
    subset_vits = subset_vits.transpose()
    subset_vits.to_csv(f"viz_outs/synthetic_vitssubset_{i}.csv", header=False)
    
    print("================================================================")